# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:
import glob
import sys
import warnings

import numpy as np
import torch
import torchvision
from tqdm.notebook import trange, tqdm
from tqdm.auto import tqdm, trange
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
# You may add any imports you need
import random
import PIL

In [2]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
set_random_seed(12345)

In [5]:
!wget -O dataset.zip https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip?dl=0
!unzip -qq dataset.zip -d dataset

--2021-11-22 13:39:37--  https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.68.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/33l8lp62rmvtx40/dataset.zip [following]
--2021-11-22 13:39:38--  https://www.dropbox.com/s/raw/33l8lp62rmvtx40/dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc85dc770d6aeb8b6d81c58533da.dl.dropboxusercontent.com/cd/0/inline/BackAIUjS9eXI9Q3eLkvbNzsogYvWv9PZPf1kTMF_72MhEr3cLGzb91jH30Wh1SDljtsHJZct5e3So-pdg3BuoJhGaCEbXYuLDlytUp3USerMHc2FXMi81x3zUyxSo35WGEowHDaF2nTlw-Nwud6heuE/file# [following]
--2021-11-22 13:39:38--  https://uc85dc770d6aeb8b6d81c58533da.dl.dropboxusercontent.com/cd/0/inline/BackAIUjS9eXI9Q3eLkvbNzsogYvWv9PZPf1kTMF_72MhEr3cLGzb91jH30Wh1SDljtsHJZct5e3So-pdg3Bu

**Первое задание (без предобученных моделей и ресайза)**

### Подготовка данных

In [6]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Normalize, Resize, ToTensor, ColorJitter, RandomHorizontalFlip, RandomRotation  

transform = Compose(
    [
          ToTensor(),
          Normalize((0.5, 0.5, 0.5), (1, 1, 1)),
          ColorJitter(hue=.05, saturation=.05),
          RandomHorizontalFlip(),
          RandomRotation(20, resample=PIL.Image.BILINEAR)
    ]
)

train_dataset = ImageFolder(
    "dataset/dataset/dataset/train", 
    transform=transform
)

val_dataset = ImageFolder(
    "dataset/dataset/dataset/val", 
    transform=transform

)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1249: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


In [7]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [8]:
def train_one_epoch(
    model,
    data_loader,
    optimizer,
    criterion,
    return_losses=False,
    device="cuda:0",
):
    model = model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    total_predictions = np.array([])#.reshape((0, ))
    total_labels = np.array([])#.reshape((0, ))
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            # Move Batch to GPU
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            # Update weights
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            # Update descirption for tqdm
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item()*100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    if return_losses:
        return metrics, all_losses
    else:
        return metrics


def validate(model, data_loader, criterion, device="cuda:0"):
    model = model.eval()
    total_loss = 0
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item()*100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    return metrics, total_predictions, total_labels

In [9]:
def train(
    model,
    epochs,
    train_data_loader,
    validation_data_loader,
    optimizer,
    criterion,
    device="cuda:0"
):
    all_train_losses = []
    epoch_train_losses = []
    epoch_eval_losses = []
    for epoch in range(epochs):
        # Train step
        print(f"Train Epoch: {epoch}")
        train_metrics, one_epoch_train_losses = train_one_epoch(
            model=model,
            data_loader=train_data_loader,
            optimizer=optimizer,
            return_losses=True,
            criterion=criterion,
            device=device
        )
        # Save Train losses
        all_train_losses.extend(one_epoch_train_losses)
        epoch_train_losses.append(train_metrics["loss"])
        # Eval step
        print(f"Validation Epoch: {epoch}")
        with torch.no_grad():
            validation_metrics = validate(
                model=model,
                data_loader=validation_data_loader,
                criterion=criterion
            )[0]
        # Save eval losses
        epoch_eval_losses.append(validation_metrics["loss"])

### Обучение модели, запуски экспериментов

In [10]:
from torchvision.models import resnet18

# Загрузить предобученную сеть -- pretrained=True
my_model = resnet18(pretrained=False)
my_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#my_model.fc = nn.Linear(512,200)
#criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(my_model.parameters(), 1e-3)
#n_epochs = 20

In [ ]:
#my_model = nn.Sequential(
#    nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5),   
#    nn.ReLU(),                                                  
#    nn.MaxPool2d(kernel_size=4),
#    nn.Flatten(),
#    nn.Linear(2250, 1024),
#    nn.ReLU(),
#    nn.Linear(1024, 200),
#)

#optimizer = torch.optim.SGD(my_model.parameters(), lr=1e-3)
#criterion = nn.CrossEntropyLoss()

In [ ]:
#my_model = nn.Sequential(
#    nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5),   
#    nn.ReLU(),                                                  
#    nn.MaxPool2d(kernel_size=4),
#    nn.Flatten(),
#    nn.Linear(2250, 128),
#    nn.ReLU(),
#    nn.BatchNorm1d(128),
#    nn.Dropout(0.1),
#    nn.Linear(128, 200),
#)

#my_model.to(device)
#optimizer = torch.optim.SGD(my_model.parameters(), lr=0.001) ######this

In [ ]:
#my_model = nn.Sequential(
#    nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5),   # добавим сверточный слой с 10 ядрами
#    nn.ReLU(),                                                  # нелинейность
#    nn.MaxPool2d(kernel_size=2),                                # уменьшим картинку в 2 раза
#    nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5),  # добавим сверточный слой с 20 ядрами
#    nn.ReLU(),                   # нелинейность
#    nn.MaxPool2d(kernel_size=2), # уменьшим картинку в 2 раза
#    nn.Flatten(),                # превращаем картинку 4х4х20 в вектор размером 320
#    nn.Linear(3380, 1024),      # линейный слой, преобразующий вектор размера 320 в вектор размера 128
#    nn.ReLU(),                   # нелинейность
#    nn.Linear(1024, 200),          # отображение в пространство рамерности числа меток классификации
#)
#my_model.to(device)
#optimizer = torch.optim.SGD(my_model.parameters(), lr=0.01) #now this

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(my_model.parameters(), 1e-3)

Простой тест на проверку правильности написанного кода

In [13]:
all_losses, predicted_labels, true_labels = validate(my_model, val_dataloader, criterion, device) #первое задание
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

  0%|          | 0/79 [00:00<?, ?it/s]

tests passed


Запустить обучение можно в ячейке ниже.

In [21]:
train(my_model, 3, train_dataloader, val_dataloader, optimizer, criterion, device)

Train Epoch: 0


  0%|          | 0/782 [00:00<?, ?it/s]

Validation Epoch: 0


  0%|          | 0/79 [00:00<?, ?it/s]

Train Epoch: 1


...

  0%|          | 0/79 [00:00<?, ?it/s]

Train Epoch: 2


  0%|          | 0/782 [00:00<?, ?it/s]

Validation Epoch: 2


  0%|          | 0/79 [00:00<?, ?it/s]

In [22]:
my_model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[ 2.7824e-03, -2.4048e-02, -6.2560e-02,  ...,  1.8283e-02,
                          6.4872e-02,  7.4909e-02],
                        [-6.6372e-02, -7.2674e-02, -5.1415e-02,  ...,  2.5392e-02,
                          5.4760e-02,  6.4677e-02],
                        [-6.1159e-02, -9.5475e-02, -4.9937e-02,  ...,  2.7196e-02,
                          8.5120e-02,  8.4802e-02],
                        ...,
                        [-5.8009e-02, -5.3379e-02, -6.4483e-02,  ...,  5.8173e-03,
                          4.1510e-02,  6.4456e-02],
                        [-4.4491e-02, -5.2062e-02, -4.1701e-02,  ..., -4.8807e-02,
                          3.4426e-02,  2.6030e-02],
                        [-3.7954e-03, -1.2752e-02, -4.4824e-03,  ..., -8.0740e-04,
                          1.7939e-02,  1.7393e-02]],
              
                       [[ 4.4525e-03,  2.6623e-02,  6.2644e-02,  ..., -5.0782e-02,
                         -6.3127

### Проверка полученной accuracy

После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, реализуйте и запустите функцию `evaluate`. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

In [23]:
all_losses, predicted_labels, true_labels = validate(my_model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за первое задание составит {} баллов".format(min(10, 10 * accuracy / 0.44)))

  0%|          | 0/79 [00:00<?, ?it/s]

Оценка за первое задание составит 7.0181818181818185 баллов


Получила лучшее качество на 17ой эпохе - 68.75% accuracy на валидации

**Второе задание**

### Подготовка данных


In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Normalize, Resize, CenterCrop, ToTensor, ColorJitter, RandomHorizontalFlip, RandomRotation  

transform = Compose(
    [
          Resize((224, 224)),
          CenterCrop((200, 200)),
          ToTensor(),
          Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
          ColorJitter(hue=.05, saturation=.05),
          RandomHorizontalFlip(),
          RandomRotation(20, resample=PIL.Image.BILINEAR),
    ]
)

resized_train = ImageFolder(
    "dataset/dataset/dataset/train", 
    transform=transform
)

resized_val = ImageFolder(
    "dataset/dataset/dataset/val", 
    transform=transform
)

resized_train_dataloader = torch.utils.data.DataLoader(resized_train, batch_size=64, shuffle=True)
resized_val_dataloader = torch.utils.data.DataLoader(resized_val, batch_size=64, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1249: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


In [ ]:
# Just very simple sanity checks
assert isinstance(resized_train[0], tuple)
assert len(resized_train[0]) == 2
assert isinstance(resized_train[1][1], int)
print("tests passed")

tests passed


### Обучение модели, запуски экспериментов

In [ ]:
from torchvision.models import resnet18

# Загрузить предобученную сеть -- pretrained=True
model = resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc = nn.Linear(512,200)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), 1e-3)
n_epochs = 20
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Простой тест на проверку правильности написанного кода

In [ ]:
all_losses, predicted_labels, true_labels = validate(model, resized_val_dataloader, criterion, device) #второе задание
assert len(predicted_labels) == len(resized_val)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

  0%|          | 0/157 [00:00<?, ?it/s]

tests passed


Запускаем обучение модели для второго задания

In [ ]:
train(model, n_epochs, resized_train_dataloader, resized_val_dataloader, optimizer, criterion, device)

Train Epoch: 0


  0%|          | 0/1563 [00:00<?, ?it/s]

Validation Epoch: 0


  0%|          | 0/157 [00:00<?, ?it/s]

Train Epoch: 1


...

  0%|          | 0/157 [00:00<?, ?it/s]

Train Epoch: 6


  0%|          | 0/1563 [00:00<?, ?it/s]

Validation Epoch: 6


  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
model.state_dict()

In [ ]:
all_losses, predicted_labels, true_labels = validate(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за второе задание составит {} баллов".format(min(10, 10 * (accuracy - 0.5) / 0.3)))

  0%|          | 0/79 [00:00<?, ?it/s]

Оценка за второе задание составит -2.106666666666666 баллов


In [ ]:
accuracy

0.4368

### Отчёт об экспериментах 

Все вспомогательные функции я взяла из семинара 5, внеся необходимые (минимальные) изменения.

При выполнении первого задания я пыталась писать свои модели (оставила их в коде в виде комментариев) - все они не давали качества выше 0.2 accuracy. Также экспериментировала с оптимайзерами и критериями - они не сильно улучшали качество, поэтому я не акцентировалась на них. В итоге я использовала модель resnet18 с семинара, отключив предобучение.

В процессе работы над вторым заданием я пробовала разные модели из torchvision (googlenet, resnet18, alexnet, vgg16...), заменяя у каждой последний слой на линейный полносвязный. То есть сначала я брала конкретную модель, подбирала для нее оптимайзер (брала только Ada и CrossEntropy) и критерий - больше ничего не меняла. Так, среди всех предобученных моделей я выбрала AlexNet 